In [20]:
!git clone https://github.com/pthengtr/kcw-analytics.git

fatal: destination path 'kcw-analytics' already exists and is not an empty directory.


In [21]:
!cd /content/kcw-analytics && git pull origin main

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 4 (delta 3), reused 0 (delta 0), pack-reused 0 (from 0)
Unpacking objects: 100% (4/4), 2.20 KiB | 375.00 KiB/s, done.
From https://github.com/pthengtr/kcw-analytics
 * branch            main       -> FETCH_HEAD
   05316b9..901ff6c  main       -> origin/main
Updating 05316b9..901ff6c
Fast-forward
 .../30_nonvat_sales_vat_purchase_report.ipynb      | 191 +++++++++++----------
 1 file changed, 100 insertions(+), 91 deletions(-)


In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
import os
import pandas as pd

folder = "/content/drive/Shareddrives/KCW-Data/kcw_analytics/01_raw"

data = {}

for file in os.listdir(folder):
    if file.endswith(".csv"):
        path = os.path.join(folder, file)
        data[file] = pd.read_csv(
            path,
            dtype={
              "BCODE": "string",
              "ITEMNO": "string",
              "BILLNO": "string",
            },
            encoding="utf-8-sig",
            low_memory=False   # stops chunk guessing
        )
        print(f"Loaded: {file} -> {data[file].shape}")

Loaded: raw_inventory_hq_2024.csv -> (4983, 8)
Loaded: raw_syp_pidet_purchase_lines.csv -> (27233, 41)
Loaded: raw_syp_pimas_purchase_bills.csv -> (2915, 49)
Loaded: raw_syp_simas_sales_bills.csv -> (12299, 49)
Loaded: raw_syp_sidet_sales_lines.csv -> (36360, 38)
Loaded: raw_hq_icmas_products.csv -> (114903, 94)
Loaded: raw_hq_pidet_purchase_lines.csv -> (153589, 41)
Loaded: raw_hq_pimas_purchase_bills.csv -> (50176, 49)
Loaded: raw_hq_sidet_sales_lines.csv -> (732969, 38)
Loaded: raw_hq_pvmas_notes_vouchers.csv -> (13730, 32)
Loaded: raw_hq_simas_sales_bills.csv -> (275965, 49)
